<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Day%2027%23%20COVID19_NetworkX_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook used to analyze the Twitter social network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import gzip
import json
from datetime import datetime
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter
#handling plots
import matplotlib.pyplot as plt
import seaborn as sns

#for network creation
import networkx as nx

In [ ]:
cd /content/drive/My Drive/"SPARK+AI Hackathon 2020"/"SPARK-2020-05"

/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05


In [ ]:
zip_file_path = os.getcwd()
zip_file_path
file_list = os.listdir(zip_file_path)
abs_path = []
count = 0
for a in file_list:
    x = zip_file_path+'/'+a
    abs_path.append(x)

In [ ]:
abs_path[:5]

['/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-05-02.jsonl.gz',
 '/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-05-08.jsonl.gz',
 '/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-07-07.jsonl.gz',
 '/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-08-10.jsonl.gz',
 '/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-06-10.jsonl.gz']

In [ ]:
count = 0
lno = 0
for f in abs_path:
    if count == 0:
        ext = os.path.splitext(f)[-1].lower()
        if ext == '.gz':
            print(f)
            with gzip.open(f, "rb") as gf:
                #for line in gf:
                  #print(lno)
                  #lno = lno+1
                  #tweet = json.loads(line)
                tweet = pd.read_json(gf,lines=True)
            count = count + 1

/content/drive/My Drive/SPARK+AI Hackathon 2020/SPARK-2020-05/coronavirus-tweet-id-2020-05-05-02.jsonl.gz


In [ ]:
tweet.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'quoted_status', 'retweet_count', 'favorite_count', 'favorited',
       'retweeted', 'possibly_sensitive', 'lang', 'extended_entities',
       'retweeted_status', 'withheld_in_countries', 'withheld_scope'],
      dtype='object')

In [ ]:
tweet.shape

(45063, 33)

In [ ]:
tweets_final = pd.DataFrame(columns = ["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                      "retweeted_id", "retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id", 
                                       "full_text", "user_id", "screen_name", "followers_count"])

In [ ]:
# Columns that are going to be the same
equal_columns = ["created_at", "id", "full_text"]
tweets_final[equal_columns] = tweet[equal_columns]

In [ ]:
def get_basics(tweets_final):
    tweets_final["screen_name"] = tweet['user'].apply(lambda x: x['screen_name'])
    tweets_final["user_id"] = tweet["user"].apply(lambda x: x["id"])
    tweets_final["followers_count"] = tweet["user"].apply(lambda x: x["followers_count"])
    return tweets_final

In [ ]:
# Get the user mentions 
def get_usermentions(tweets_final):
    
    tweets_final["user_mentions_screen_name"] = tweet["entities"].apply(lambda x: x["user_mentions"][0]["screen_name"] if x["user_mentions"] else np.nan)
    tweets_final["user_mentions_id"] = tweet["entities"].apply(lambda x: x["user_mentions"][0]["id_str"] if x["user_mentions"] else np.nan)
    return tweets_final

In [ ]:
def get_retweets(tweets_final):
     
    tweets_final["retweeted_screen_name"] = tweet["retweeted_status"].apply(lambda x: x["user"]["screen_name"] if x is not np.nan else np.nan)
    tweets_final["retweeted_id"] = tweet["retweeted_status"].apply(lambda x: x["user"]["id_str"] if x is not np.nan else np.nan)
    return tweets_final

In [ ]:
def get_in_reply(tweets_final):
    tweets_final["in_reply_to_screen_name"] = tweet["in_reply_to_screen_name"]
    tweets_final["in_reply_to_status_id"] = tweet["in_reply_to_status_id"]
    tweets_final["in_reply_to_user_id"]= tweet["in_reply_to_user_id"]
    return tweets_final

In [ ]:
def fill_df(tweets_final):
    get_basics(tweets_final)
    get_usermentions(tweets_final)
    get_retweets(tweets_final)
    get_in_reply(tweets_final)
    return tweets_final

In [ ]:
# Get the interactions between the different users
def get_interactions(row):
    
    user = row["user_id"], row["screen_name"]
    if user[0] is None:
        return (None, None), []
    
    interactions = set()
    
    
    interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
    interactions.add((row["retweeted_id"], row["retweeted_screen_name"]))
    interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))
    
    interactions.discard((row["user_id"], row["screen_name"]))
    interactions.discard((None, None))
    return user, interactions

In [ ]:
tweets_final = fill_df(tweets_final)

In [ ]:
tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

In [ ]:
tweets_final.head(5)

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweeted_id,retweeted_screen_name,user_mentions_screen_name,user_mentions_id,full_text,user_id,screen_name,followers_count
0,2020-05-05 02:03:19+00:00,1257491031630786561,None,None,None,None,None,None,None,Muslims must live by American laws or they mus...,986630456849453057,DreamerSeriespu,1602
1,2020-05-05 02:03:19+00:00,1257491031777370118,AndrewScheer,1.25747e+18,2.56361e+08,None,None,AndrewScheer,256360738,@AndrewScheer Trudeau announces more investmen...,4499430739,Learning2GetBy,6691
2,2020-05-05 02:03:19+00:00,1257491031794253824,clayden_luke,1.25749e+18,1.22944e+18,None,None,None,None,As I suspected #covid19 #coronavirus https://t...,1229440370544717824,clayden_luke,55
3,2020-05-05 02:03:19+00:00,1257491031806754816,None,None,None,67109577,BrainHarrington,BrainHarrington,67109577,RT @BrainHarrington: Lol they are just making ...,20884752,snadp,1303
4,2020-05-05 02:03:19+00:00,1257491031945338883,None,None,None,None,None,None,None,Nothing about protecting corporations from bei...,3170869955,debmacp1,2305


In [ ]:
graph = nx.Graph()

In [ ]:
for index, tweets in tweets_final.iterrows():
    user, interactions = get_interactions(tweets)
    user_id, user_name = user
    tweet_id = tweets["id"]
    #tweet_sent = tweet["sentiment"]
    for interaction in interactions:
        int_id, int_name = interaction
        graph.add_edge(user_id, int_id, tweet_id=tweet_id)
        

        graph.nodes[user_id]["name"] = user_name
        graph.nodes[int_id]["name"] = int_name

In [ ]:
print(f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph")

There are 48637 nodes and 39943 edges present in the Graph


In [ ]:
degrees = [val for (node, val) in graph.degree()]

In [ ]:
print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")

The maximum degree of the Graph is 593
The minimum degree of the Graph is 1


In [ ]:
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)[0][0]}")

The average degree of the nodes in the Graph is 1.6
The most frequent degree of the nodes found in the Graph is 1


In [ ]:
if nx.is_connected(graph):
    print("The graph is connected")
else:
    print("The graph is not connected")

The graph is not connected


In [ ]:

print(f"There are {nx.number_connected_components(graph)} connected components in the Graph")

There are 10818 connected components in the Graph


In [ ]:
largest_subgraph = max((graph.subgraph(c) for c in nx.connected_components(graph)), key=len)

In [ ]:
print(f"There are {largest_subgraph.number_of_nodes()} nodes and {largest_subgraph.number_of_edges()} \
edges present in the largest component of the Graph")

There are 14499 nodes and 16274 edges present in the largest component of the Graph


In [ ]:
if nx.is_connected(largest_subgraph):
    print("The graph is connected")
else:
    print("The graph is not connected")

The graph is connected


In [ ]:

print(f"The average clustering coefficient is {nx.average_clustering(largest_subgraph)} in the largest subgraph")
print(f"The transitivity of the largest subgraph is {nx.transitivity(largest_subgraph)}")

The average clustering coefficient is 0.0 in the largest subgraph
The transitivity of the largest subgraph is 0


In [ ]:
print(f"The diameter of our Graph is {nx.diameter(largest_subgraph)}")
print(f"The average distance between any two nodes is {nx.average_shortest_path_length(largest_subgraph):.2f}")

The diameter of our Graph is 37
The average distance between any two nodes is 9.69


In [ ]:
graph_centrality = nx.degree_centrality(largest_subgraph)

In [ ]:
max_de = max(graph_centrality.items(), key=itemgetter(1))

In [ ]:
graph_closeness = nx.closeness_centrality(largest_subgraph)

In [ ]:
max_clo = max(graph_closeness.items(), key=itemgetter(1))

In [ ]:
graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)

In [ ]:
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

In [ ]:
print(f"the node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph")

the node with id 25073877 has a degree centrality of 0.04 which is the maximum of the Graph
the node with id 25073877 has a closeness centrality of 0.16 which is the maximum of the Graph
the node with id 25073877 has a betweenness centrality of 0.28 which is the maximum of the Graph


In [ ]:
node_and_degree = largest_subgraph.degree()
colors_central_nodes = ['orange', 'red']
central_nodes = ['28785486', '267695291']

In [ ]:
pos = nx.spring_layout(largest_subgraph, k=0.05)

In [ ]:
plt.figure(figsize = (20,20))
nx.draw(largest_subgraph, pos=pos, node_color=range(5927), cmap=plt.cm.PiYG, edge_color="black", linewidths=0.3, node_size=60, alpha=0.6, with_labels=False)
nx.draw_networkx_nodes(largest_subgraph, pos=pos, nodelist=central_nodes, node_size=300, node_color=colors_central_nodes)
plt.savefig('graphfinal.png')
plt.show()